## Pandas的索引index的用途

把数据存储于普通的column列也能用于数据查询，那使用index有什么好处？

index的用途总结：  
1. 更方便的数据查询；
2. 使用index可以获得性能提升；
3. 自动的数据对齐功能；
4. 更多更强大的数据结构支持；


In [1]:
import pandas as pd

In [28]:
df = pd.read_csv("./datas/ml-latest-small/ratings.csv")

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

## 1、使用index查询数据

In [5]:
# drop==False，让索引列还保持在column，如果不设置drop=false,则数据体中userid就消失了
df.set_index("userId", inplace=True, drop=False)

In [6]:
df.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


In [29]:
#df.set_index("userId", inplace=True, drop=True)
#df.head()

,movieId,rating,timestamp
userId,,,
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931


In [7]:
df.index

Int64Index([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
            ...
            610, 610, 610, 610, 610, 610, 610, 610, 610, 610],
           dtype='int64', name='userId', length=100836)

In [25]:
df.values

array([[1.00000000e+00, 1.00000000e+00, 4.00000000e+00, 9.64982703e+08],
       [1.00000000e+00, 3.00000000e+00, 4.00000000e+00, 9.64981247e+08],
       [1.00000000e+00, 6.00000000e+00, 4.00000000e+00, 9.64982224e+08],
       ...,
       [6.10000000e+02, 1.68250000e+05, 5.00000000e+00, 1.49427305e+09],
       [6.10000000e+02, 1.68252000e+05, 5.00000000e+00, 1.49384635e+09],
       [6.10000000e+02, 1.70875000e+05, 3.00000000e+00, 1.49384642e+09]])

In [8]:
# 使用index的查询方法
df.loc[500].head(5)

,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


In [9]:
# 使用column的condition查询方法
df.loc[df["userId"] == 500].head()

,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


## 2. 使用index会提升查询性能

* 如果index是唯一的，Pandas会使用哈希表优化，查询性能为O(1);查一次
* 如果index不是唯一的，但是有序，Pandas会使用二分查找算法，查询性能为O(logN);
* 如果index是完全随机的，那么每次查询都要扫描全表，查询性能为O(N);遍历N行
* 所以建立索引之后最好排序。

<img src="./other_files/pandas-index-performance.png" width=600/>

### 实验1：完全随机的顺序查询

In [10]:
# 将数据随机打散
from sklearn.utils import shuffle
df_shuffle = shuffle(df)

In [11]:
df_shuffle.head()

,userId,movieId,rating,timestamp
userId,,,,
175,175,7169,5.0,1234189772
365,365,89745,2.5,1488332569
6,6,236,4.0,845553559
389,389,805,5.0,857934470
517,517,38038,2.5,1487955853


In [12]:
# 索引是否是递增的
df_shuffle.index.is_monotonic_increasing

False

In [13]:
df_shuffle.index.is_unique

False

In [14]:
# 计时，查询id==500数据性能
%timeit df_shuffle.loc[500]

955 µs ± 184 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 实验2：将index排序后的查询

In [15]:
df_sorted = df_shuffle.sort_index()

In [16]:
df_sorted.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,2648,4.0,964983414
1,1,2046,4.0,964982563
1,1,2018,5.0,964980523
1,1,3448,5.0,964984054
1,1,157,5.0,964984100


In [17]:
# 索引是否是递增的
df_sorted.index.is_monotonic_increasing

True

In [18]:
df_sorted.index.is_unique

False

In [19]:
%timeit df_sorted.loc[500]

418 µs ± 8.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 3. 使用index能自动对齐数据

包括series和dataframe

In [20]:
s1 = pd.Series([1,2,3], index=list("abc"))

In [21]:
s1

a    1
b    2
c    3
dtype: int64

In [22]:
s2 = pd.Series([2,3,4], index=list("bcd"))

In [23]:
s2

b    2
c    3
d    4
dtype: int64

In [24]:
s1+s2

a    NaN
b    4.0
c    6.0
d    NaN
dtype: float64